<a href="https://colab.research.google.com/github/craravenav/apiferremas/blob/main/apiIntegracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Configuraciones de Python flask y ngrok (serv web)
!pip install flask-ngrok
!pip install pyngrok
# Token ngrok NO TOCAR
!ngrok authtoken "2fyigyJ5QiOpjzoUequD64Hh76C_5Qprcn23nwXrrruLZCE9y"
!ngrok config upgrade

# Descarga Driver SQL Server 18
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo tee /etc/apt/trusted.gpg.d/microsoft.asc

!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
# Actualiza Gestor de Packetes Linux e instala SQL Server ODBC
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get -q -y install odbcinst
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18

# Instala libreria python para ODBC
!pip install pyodbc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
No changes needed for config file: '/root/.config/ngrok/ngrok.yml'
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   6621      0 --:--:-- --:--:-- --:--:--  6641
-----BEGIN PGP PUBLIC KEY BLOCK-----
Version: GnuPG v1.4.7 (GNU/Linux)

mQENBFYxWIwBCADAKoZhZlJxGNGWzqV+1OG1xiQeoowKhssGAKvd+buXCGISZJwT
LXZqIcIiLP7pqdcZWtE9bSc7yBY2MalDp9Liu0KekywQ6VVX1T72NPf5Ev6x6DLV
7aVWsCzUAF+eb7DC9fPuFLEdxmOEYoPjzrQ7cCnSV4JQxAqhU4T6OjbvRazGl3ag
OeizPXmRljMtUUttHQZnRhtlzkmwIrUivbfFPD+fEoHJ1+uIdfOzZX8/oKHKLe2j
H632kvsNzJFlROVvGLYAk2WRcLu+RjjggixhwiB+Mu/A8Tf4V6b+YppS44q8EvVr
M+QvY7LNSOffSO6Slsy9oisGTdfE39nC7pVRABEBAAG0N01pY3Jvc29mdCAoUmVs
ZWFzZSBzaWduaW5nKSA8Z3Bnc2VjdXJpdHlAbWljcm9zb2Z0LmNvbT6JATUEEwEC
AB8FAlYxWIwCGwMGCwkIBwMCBBUCCAMDFgIBAh4BAheAAAoJEOs+lK2+EinPGpsH
/32vKy29Hg51H9dfFJMx0/a/F+5vKeCeVqi

In [ ]:
!curl ipecho.net/plain

35.196.211.245

In [10]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import json, requests
import pyodbc

############################################################################################################################################################
# CONFIGURACION AZURE SQL SERVER NO TOCAR
############################################################################################################################################################
SERVER = 'srvferremas.database.windows.net' # Server example
DATABASE = 'ferremas'
USERNAME = 'ferremasadmin'
PASSWORD = 'Integracion.2024'
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};Server=tcp:{SERVER},1433;Database={DATABASE};Uid={USERNAME};Pwd={PASSWORD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'

############################################################################################################################################################
#Inicio App con Flask
app = Flask(__name__)
run_with_ngrok(app)

# API Inicio Home
@app.route("/")
def home():
  return "Estamos Online!!"

############################################################################################################################################################
# API SQL Server AZURE CLOUD
############################################################################################################################################################
# Descripcion: Definición de CRUD para tabla producto
############################################################################################################################################################

# Add Producto recibe json con datos del producto que se desea agregar
############################################################################################################################################################
@app.route("/producto", methods=['POST'])
def add_producto():

    #Inicia Conexión
    connection = pyodbc.connect(connectionString)
    #Inicia Cursor
    cursor = connection.cursor()

    # CAPTURA DE PARAMETROS
    # Se obtiene json con datos para actualizar
    data = request.get_json()

    codigo_producto = data.get('codigo_producto')
    marca = data.get('marca')
    codigo = data.get('codigo')
    nombre = data.get('nombre')
    categoria = data.get('categoria')
    # LOGICA SQL para agregar productos



    # Cierra Conexión
    connection.close()
    return "Hola agregare un producto"
############################################################################################################################################################

# Get Producto acepta /all /id?= /marca?=
############################################################################################################################################################
@app.route("/producto", methods=['GET'])
def get_producto():

    #Inicia Conexión a SQL
    connection = pyodbc.connect(connectionString)
    #Inicia Cursor
    cursor = connection.cursor()

    # CAPTURA DE PARAMETROS
    # obtiene todos los Productos
    if 'all' in request.args:
        cursor.execute('SELECT [codigo_producto],[marca],[codigo_marca],[nombre],[categoria] FROM dbo.producto')
    # obtiene solo el Producto del id solicitado
    elif 'id' in request.args:
        id = request.args.get('id')
        cursor.execute('SELECT [codigo_producto],[marca],[codigo_marca],[nombre],[categoria] FROM dbo.producto WHERE codigo_producto = ?', id)
    # obtiene todos los Productos de una Marca solicitada
    elif 'marca' in request.args:
        marca = request.args.get('marca')
        cursor.execute('SELECT [codigo_producto],[marca],[codigo_marca],[nombre],[categoria] FROM dbo.producto WHERE marca = ?', marca)
    # Si se invoca cualquier otro metodo de producto lanza error
    else:
        return jsonify({'error': 'Parámetro incorrecto'}), 400

    # Obtiene los Registros
    data = cursor.fetchall()
    # Cierra el Cursor
    cursor.close()
    # Cierra Conexión
    connection.close()

    # Formatea los resultados en JSON
    json_data = []
    for row in data:
        table_row = {
            'Codigo del producto': row[0],
            'Marca': row[1],
            'Codigo Marca': row[2],
            'Nombre del Producto': row[3],
            'Categoria': row[4]
        }
        json_data.append(table_row)

    return jsonify(json_data)
############################################################################################################################################################

# Update Producto recibe json con atributos de la tabla producto
############################################################################################################################################################
@app.route('/producto', methods=['PUT'])
def update_producto():
    try:
        # Se obtiene json con datos para actualizar
        data = request.get_json()
        # Se valida si vienen datos en el json sino envia error
        if not data:
            return jsonify({'error': 'No se proporcionaron datos para actualizar'}), 400

        # Se obtiene el codigo del producto
        id = data.get('codigo_producto')
        # En caso de que no venga el codigo del producto envia error
        if not id:
            return jsonify({'error': 'Se necesita un ID para actualizar los datos'}), 400

        # Verificar si el ID existe en la base de datos
        #Inicia Conexión a SQL
        connection = pyodbc.connect(connectionString)
        #Inicia Cursor
        cursor = connection.cursor()
        #Define la consulta para buscar el codigo del producto que se esta intentando actualizar
        cursor.execute('SELECT COUNT(*) FROM dbo.producto WHERE codigo_producto = ?', id)
        #Asigna el recuento a la variable count para saber cuantos productos existen
        count = cursor.fetchone()[0]
        # Cierra el cursor
        cursor.close()
        # Cierra Conexión
        connection.close()

        # Verifica si el ID del producto fue encontrado en la base de datos
        if count == 0:
            return jsonify({'error': 'El ID del producto no existe'}), 404


        # Logica que ejecuta la actualización en la base de datos del producto
        #Inicia Conexión a SQL
        connection = pyodbc.connect(connectionString)
        #Inicia Cursor
        cursor = connection.cursor()
        #Define la Consulta para actualizar la tabla producto [codigo_producto],[marca],[codigo_marca],[nombre],[categoria]
        cursor.execute('UPDATE dbo.producto SET [marca]=?, [codigo_marca]=?, [nombre]=?, [categoria]=?  WHERE [codigo_producto]=?', (data['marca'], data['codigo_marca'], data['nombre'], data['categoria'], id))
        #Realiza el commit de la actualización para confirmar los cambios
        connection.commit()
        # Cierra el cursor
        cursor.close()
        # Cierra Conexión
        connection.close()

        # Responde correctamente que se ingreso el producto
        return jsonify({'mensaje': 'Producto actualizado correctamente'}), 200
    except Exception as e:
        return jsonify({'error': 'Se produjo un error al actualizar los datos: {}'.format(str(e))}), 500
############################################################################################################################################################

# Delete Producto recibe json con atributos de la tabla producto
############################################################################################################################################################
@app.route('/producto', methods=['DELETE'])
def delete_producto():
    return "hola voy a borrar"
############################################################################################################################################################

############################################################################################################################################################
# API Banco Central
############################################################################################################################################################
# Descripcion: consulta el valor del dolar y transforma el dolar solicitado a moneda local (CLP)
############################################################################################################################################################
@app.route("/usdtoclp", methods=['GET'])
def dolar_a_peso():
    dolar = 10
    # Configuracion de Credenciales BCENTRAL
    #base_url = "https://si3.bcentral.cl/SieteRestWS/SieteRestWS.ashx"
    #user = "carl.aravena@duocuc.cl"
    #password = "R27029155-4"
    #fechaIni = "2024-05-03"
    #fechaFin = "2024-05-03"
    #timeseries = "F073.TCO.PRE.Z.D"
    #funcion = "GetSeries"

    # Obtener el parámetro 'param' de la URL
    param_dolar = request.args.get('usd')

    # Configuracion URL
    url = "https://si3.bcentral.cl/SieteRestWS/SieteRestWS.ashx?user=carl.aravena@duocuc.cl&pass=R27029155-4&firstdate=2024-05-03&lastdate=2024-05-03&timeseries=F073.TCO.PRE.Z.D&function=GetSeries"

    # Realizar la solicitud GET a la API
    response = requests.get(url)
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200 and param_dolar:
        # Convertir la respuesta JSON a un diccionario de Python
        data = response.json()
        # Obtener el valor de "value" de "obs"
        valor = data["Series"]["Obs"][0]["value"]
        clp = float(valor)*float(param_dolar)
        return jsonify(
          dolar_consultado=str(param_dolar),
          valor_clp=str(clp)
        )
        #return "Valor en CLP: "+ str(clp)
    else:
        return "Error al realizar la solicitud a la API: "+ response.status_code


ngrok_tunnel = ngrok.connect(5000)
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d6e9-35-196-66-245.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/May/2024 07:07:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 07:07:54] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 07:08:09] "PUT /producto HTTP/1.1" 404 -
